In [1]:
import requests
import time
import pandas as pd

In [2]:
from dotenv import load_dotenv
import os

load_dotenv()  # loads variables from .env into environment

mapbox_api_key = os.getenv('MAPBOX_API_KEY')
census_api_key = os.getenv('CENSUS_API_KEY')

print(mapbox_api_key)  # Just to check it loads


pk.eyJ1IjoiY2Fycmllcm9iaW4iLCJhIjoiY21jM2ltdTlyMDU1NTJrb2ZnMnFiazRqeiJ9.RT-SWZnQIo_Z7zSOdRh_vg


In [ ]:
#make the Census Bureau API call

VARIABLES = [
  "NAME",
  "B02001_001E","B02001_002E","B02001_003E","B02001_004E",
  "B02001_005E","B02001_006E","B02001_007E","B02001_008E",
  "B01001_002E","B01001_026E",
  "B19013_001E",
  "B17001_001E","B17001_002E"
]

response = requests.get(
    "https://api.census.gov/data/2022/acs/acs5",
    params={
      "get": ",".join(VARIABLES),
      "for": "county:*",
      "key": CENSUS_API_KEY
    }
)

data = response.json()
df = pd.DataFrame(data[1:], columns=data[0])
df.to_csv("acs2022_race_age_income_poverty_counties.csv", index=False)


In [33]:
acs2022 = pd.read_csv("acs2022_race_age_income_poverty_counties.csv")
acs2022.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3222 entries, 0 to 3221
Data columns (total 16 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   NAME         3222 non-null   object
 1   B02001_001E  3222 non-null   int64 
 2   B02001_002E  3222 non-null   int64 
 3   B02001_003E  3222 non-null   int64 
 4   B02001_004E  3222 non-null   int64 
 5   B02001_005E  3222 non-null   int64 
 6   B02001_006E  3222 non-null   int64 
 7   B02001_007E  3222 non-null   int64 
 8   B02001_008E  3222 non-null   int64 
 9   B01001_002E  3222 non-null   int64 
 10  B01001_026E  3222 non-null   int64 
 11  B19013_001E  3222 non-null   int64 
 12  B17001_001E  3222 non-null   int64 
 13  B17001_002E  3222 non-null   int64 
 14  state        3222 non-null   int64 
 15  county       3222 non-null   int64 
dtypes: int64(15), object(1)
memory usage: 402.9+ KB


In [34]:
import requests
import pandas as pd
import time

API_KEY = "e8efa7897eff888afb8549682189c87625f9aabb"

VARIABLES = [
    "NAME",
    "B02001_001E","B02001_002E","B02001_003E","B02001_004E",
    "B02001_005E","B02001_006E","B02001_007E","B02001_008E",
    "B01001_002E","B01001_026E",
    "B19013_001E",
    "B17001_001E","B17001_002E"
]

# All 50 states + DC (FIPS codes 01–56, skipping 3 U.S. territories)
state_fips = [f"{i:02d}" for i in range(1, 57) if i not in [3, 7, 14, 43, 52]]  # exclude territories

all_data = []

for state in state_fips:
    print(f"Fetching data for state {state}...")
    try:
        r = requests.get(
            "https://api.census.gov/data/2022/acs/acs5",
            params={
                "get": ",".join(VARIABLES),
                "for": "tract:*",
                "in": f"state:{state} county:*",
                "key": API_KEY
            }
        )
        r.raise_for_status()
        data = r.json()
        df = pd.DataFrame(data[1:], columns=data[0])
        all_data.append(df)
        time.sleep(1)  # be kind to the API
    except Exception as e:
        print(f"Failed for state {state}: {e}")

# Combine all results
tracts = pd.concat(all_data, ignore_index=True)
tracts.to_csv("acs2022_all_states_tracts.csv", index=False)
print("Done.")


Fetching data for state 01...
Fetching data for state 02...
Fetching data for state 04...
Fetching data for state 05...
Fetching data for state 06...
Fetching data for state 08...
Fetching data for state 09...
Fetching data for state 10...
Fetching data for state 11...
Fetching data for state 12...
Fetching data for state 13...
Fetching data for state 15...
Fetching data for state 16...
Fetching data for state 17...
Fetching data for state 18...
Fetching data for state 19...
Fetching data for state 20...
Fetching data for state 21...
Fetching data for state 22...
Fetching data for state 23...
Fetching data for state 24...
Fetching data for state 25...
Fetching data for state 26...
Fetching data for state 27...
Fetching data for state 28...
Fetching data for state 29...
Fetching data for state 30...
Fetching data for state 31...
Fetching data for state 32...
Fetching data for state 33...
Fetching data for state 34...
Fetching data for state 35...
Fetching data for state 36...
Fetching d

PermissionError: [Errno 13] Permission denied: 'acs2022_all_states_tracts.csv'

In [ ]:
tracts["census_tract"] = (
    tracts["state"].str.zfill(2) +
    tracts["county"].str.zfill(3) +
    tracts["tract"].str.zfill(6)
)


In [ ]:
#read in csv of 2025 clinics with full census information
clinic_census=pd.read_csv("df2025_full_census.csv")
clinic_census.head(5)

,CMS Certification Number (CCN),Network,Facility Name,Five Star Date,Five Star,Five Star Data Availability Code,Address Line 1,Address Line 2,City/Town,State,...,Percentage of Adult patients with long term catheter in use,nPCR Data Availability Code,Number of patients in nPCR summary,Number of patient-months in nPCR summary,Percentage of pediatric HD patients with nPCR,dfc_new,full_address,latitude,longitude,census_tract
0,12306,8,CHILDRENS HOSPITAL DIALYSIS,01Jan2020-31Dec2023,NaN,260,1600 7TH AVENUE SOUTH,NaN,BIRMINGHAM,AL,...,NaN,1,24.0,175.0,99.0,existing,"1600 7TH AVENUE SOUTH, BIRMINGHAM, AL 35233",33.504018,-86.805215,1.073005e+09
1,12500,8,FMC CAPITOL CITY,01Jan2020-31Dec2023,1.0,1,255 S JACKSON STREET,NaN,MONTGOMERY,AL,...,16.0,259,0.0,NaN,NaN,existing,"255 S JACKSON STREET, MONTGOMERY, AL 36104",32.374048,-86.295939,1.101001e+09
2,12501,8,DaVita Gadsden Dialysis,01Jan2020-31Dec2023,2.0,1,409 SOUTH FIRST STREET,NaN,GADSDEN,AL,...,12.0,259,0.0,NaN,NaN,existing,"409 SOUTH FIRST STREET, GADSDEN, AL 35901",34.006750,-86.003132,1.055001e+09
3,12502,8,DaVita Tuscaloosa University Dialysis,01Jan2020-31Dec2023,1.0,1,220 15TH STREET,NaN,TUSCALOOSA,AL,...,11.0,259,0.0,NaN,NaN,existing,"220 15TH STREET, TUSCALOOSA, AL 35401",33.198990,-87.538230,1.125012e+09
4,12505,8,DaVita PDI-Montgomery,01Jan2020-31Dec2023,3.0,1,1001 FOREST AVENUE,NaN,MONTGOMERY,AL,...,14.0,259,0.0,NaN,NaN,existing,"1001 FOREST AVENUE, MONTGOMERY, AL 36106",32.369331,-86.285969,1.101001e+09


In [ ]:
# convert census tracts to same dtype (strings with no decimal places)

tracts["census_tract"] = tracts["census_tract"].astype(str).str.zfill(11)
clinic_census["census_tract"] = clinic_census["census_tract"].astype(str).str.zfill(11)


#merge dfs on census tract number

merged = pd.merge(tracts, clinic_census, on='census_tract') 
merged.head(5)


,CMS Certification Number (CCN),Network,Facility Name,Five Star Date,Five Star,Five Star Data Availability Code,Address Line 1,Address Line 2,City/Town,State,...,B02001_007E,B02001_008E,B01001_002E,B01001_026E,B19013_001E,B17001_001E,B17001_002E,state,county,tract


In [35]:
print(tracts['census_tract'].nunique())
print(clinic_census['census_tract'].nunique())

KeyError: 'census_tract'

In [27]:
#create percentage variables
# Avoid division by zero
merged['B02001_002E'] = merged['B02001_002E'].astype(float)
merged['B02001_001E'] = merged['B02001_001E'].astype(float)
merged['B02001_003E'] = merged['B02001_003E'].astype(float)
merged['B17001_002E'] = merged['B17001_002E'].astype(float)
merged['B17001_001E'] = merged['B17001_001E'].astype(float)

merged["pct_white"] = merged["B02001_002E"] / merged["B02001_001E"]
merged["pct_black"] = merged["B02001_003E"] / merged["B02001_001E"]
merged["pct_poverty"] = merged["B17001_002E"] / merged["B17001_001E"]


In [29]:
merged['B17001_001E'].describe()

count    0.0
mean     NaN
std      NaN
min      NaN
25%      NaN
50%      NaN
75%      NaN
max      NaN
Name: B17001_001E, dtype: float64

In [19]:
merged['pct_white'] = merged['pct_white'].astype(float)
merged['pct_black'] = merged['pct_black'].astype(float)
merged['pct_poverty'] = merged['pct_poverty'].astype(float)

print(merged)
print(merged.dtypes)

Empty DataFrame
Columns: [CMS Certification Number (CCN), Network, Facility Name, Five Star Date, Five Star, Five Star Data Availability Code, Address Line 1, Address Line 2, City/Town, State, ZIP Code, County/Parish, Telephone Number, Profit or Non-Profit, Chain Owned, Chain Organization, Late Shift, # of Dialysis Stations, Offers in-center hemodialysis, Offers peritoneal dialysis, Offers home hemodialysis training, Certification Date, Claims Date, EQRS Date, SMR Date, Patient Survival Category Text, Patient Survival data availability code, Number of Patients included in survival summary, Mortality Rate (Facility), Mortality Rate: Upper Confidence Limit (97.5%), Mortality Rate: Lower Confidence Limit (2.5%), SHR Date, Patient hospitalization category text, Patient Hospitalization data availability Code, Number of patients included in hospitalization summary, Hospitalization Rate (Facility), Hospitalization Rate: Upper Confidence Limit (97.5%), Hospitalization Rate: Lower Confidence Li

In [22]:
merged['pct_white'].describe()

count    0.0
mean     NaN
std      NaN
min      NaN
25%      NaN
50%      NaN
75%      NaN
max      NaN
Name: pct_white, dtype: float64

In [20]:
import statsmodels.api as sm

# Select predictors and outcome
X = merged[["pct_poverty", "pct_black", "pct_white"]]
X = sm.add_constant(X)  # adds the intercept term
y = merged["Five Star"]

# Fit linear regression model
model = sm.OLS(y, X, missing='drop').fit()

# Print the results
print(model.summary())


ValueError: zero-size array to reduction operation maximum which has no identity